In [9]:

# cost (type: float)

import psycopg2

def fetch_cost(sql):
    """ Execute sql and fetch the results """
    sql = "explain (FORMAT JSON) " + sql
    
    conn = psycopg2.connect(database='tpch1x', # tpch1x (0.1m, 10m), tpch100m (100m)
                            user='postgres',
                            password='postgres',
                            host='166.111.121.62',
                            port=5432)
    fail = 0
    cur = conn.cursor()
    try:
        cur.execute(sql)
    except:
        fail = 1
    res = -1
    if fail==0:
        res = cur.fetchall()
        
        conn.commit()
        cur.close()
        conn.close()
        
        res = res[0][0][0]['Plan']['Total Cost']
        return res
    else:
        print("execution fails!!")
        return res


In [13]:
# query rewrite

import json
from pathlib import Path
import os

ppath = Path().absolute()
print(ppath)

# a Python object (dict)
# ["origin query","rewritten query", "rewrite sequence"]
rewrite_set = {}
base_dir = os.path.abspath(os.curdir)


# load queries into [rewrite_set]
'''
qid = 0
with open(os.path.join(base_dir, 'sqlsmith_queries', 'l_3_queries.log'), 'r') as f:

        sql = ""
        for line in f.readlines():
            sql = sql + line
            if ';' in line:
'''
query_dir = os.path.join(base_dir, 'queries', 'input_sql')
for file in sorted(os.listdir(query_dir)):
    if not file.endswith('.sql'):
        continue

    with open(os.path.join(query_dir, file), 'r') as sql_file:
        # sqls = [sql.value for sql in sqlparse.parse(sql_file.read()) if sql.get_type() == 'SELECT']
        # sqls = []
        # for sql in sqls:
        sql = str(sql_file.read())
        origin_cost = fetch_cost(sql)            

        # query rewrite
        para = "--sql \"{}\"".format(sql)
        %run 'main.py' $para

        with open('tmp-result-query','r') as f:
            rewritten_sql = f.read()
        #os.remove("tmp-result-query")

        with open('tmp-result-rewrites','r') as f:
            rewrite_order = f.read()
        #os.remove("tmp-result-rewrites")

        rewritten_cost = fetch_cost(rewritten_sql)            
        rewrite_set.update({str(qid):{
              "origin_query": sql,
              "origin_cost": origin_cost,                
              "rewritten_query": rewritten_sql,
              "rewritten_cost": rewritten_cost,                            
              "rewrite_order": rewrite_order}})
        print("==================================")
        print(file)
        print(str({
              "origin_query": sql,
              "origin_cost": origin_cost,
              "rewritten_query": rewritten_sql,
              "rewritten_cost": rewritten_cost,
              "rewrite_order": rewrite_order}))
        print("==================================")
        sql = ""
        qid = qid + 1
        # if qid >0:
        #     break
                
# print(rewrite_set)

/Users/xuanhe/benchmark/ReWrite/ReWrite-old/part2-rewrite-calcite/LearnedRewrite
planner configured
dialect configured
level 0
Best Child: SELECT "lineitem"."l_orderkey", "orders"."o_orderdate", "orders"."o_shippriority", "lineitem"."l_extendedprice" * (1 - "lineitem"."l_discount") AS "$f3"
FROM "customer",
"orders",
"lineitem"
WHERE "customer"."c_mktsegment" = 'BUILDING' AND "customer"."c_custkey" = "orders"."o_custkey" AND "lineitem"."l_orderkey" = "orders"."o_orderkey" AND "orders"."o_orderdate" < DATE '1995-03-15' AND "lineitem"."l_shipdate" > DATE '1995-03-15' 5748.439999999944 [2, 1, 4]
--------------------------------
final sql:  SELECT "lineitem"."l_orderkey", "orders"."o_orderdate", "orders"."o_shippriority", "lineitem"."l_extendedprice" * (1 - "lineitem"."l_discount") AS "$f3"
FROM "customer",
"orders",
"lineitem"
WHERE "customer"."c_mktsegment" = 'BUILDING' AND "customer"."c_custkey" = "orders"."o_custkey" AND "lineitem"."l_orderkey" = "orders"."o_orderkey" AND "orders"."o_o

In [4]:
# query rewrite (rewritten query, cost, rewrite rules)
query = "--sql \'select * from customer;\'"
%run 'main.py' $query
# read the rewritten query from log


select * from customer;
select * from customer;
origin cost: 13168.62
planner configured
dialect configured
level 0
Best Child: SELECT *
FROM "customer" 0.0
--------------------------------
SELECT *
FROM "customer"
after-rewrite cost: 13168.62


In [35]:
# convert into JSON:
rewrite_set = json.dumps(rewrite_set)

# the result is a JSON string:
print((rewrite_set))
with open(ppath.joinpath('test_query.log'),'w') as f:
    f.write(rewrite_set)

{"0": {"origin_query": "select  \n  sample_0.ps_supplycost as c0, \n  sample_0.ps_availqty as c1, \n  case when EXISTS (\n      select  \n          subq_0.c0 as c0, \n          sample_0.ps_availqty as c1, \n          sample_1.l_returnflag as c2, \n          subq_0.c0 as c3, \n          sample_1.l_commitdate as c4, \n          85 as c5, \n          sample_0.ps_suppkey as c6\n        from \n          public.lineitem as sample_1 tablesample system (1.3) ,\n          lateral (select  \n                sample_0.ps_null as c0\n              from \n                public.nation as ref_0\n              where (cast(null as circle) && cast(null as circle)) \n                and (cast(null as aclitem) = cast(null as aclitem))\n              limit 113) as subq_0\n        where EXISTS (\n          select  \n              sample_0.ps_partkey as c0, \n              subq_0.c0 as c1, \n              sample_1.l_null as c2, \n              subq_0.c0 as c3, \n              subq_0.c0 as c4\n            fro

In [10]:
rewrite_set.append({"12":123})

AttributeError: 'str' object has no attribute 'append'

In [18]:
rewrite_set = {
"0":{  "origin": "",
  "rewritten": "",
  "rewrites": []}
}

rewrite_set.update({"2":{  "origin": "",
  "rewritten": "",
  "rewrites": []}
})
print(rewrite_set)

{'0': {'origin': '', 'rewritten': '', 'rewrites': []}, '2': {'origin': '', 'rewritten': '', 'rewrites': []}}


In [5]:
b = 'asdf'
a = "hello come here {}".format(b)
print(a)

hello come here asdf


In [22]:
a = [1,2]
b = a + [123]
print(str(a))

[1, 2]


In [31]:
a = ['sdf']
b =a[-1]
c =a[:-1]
print(b,c)

sdf []
